In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from google.colab import drive


# Define paths (adjust these paths to your dataset location)
dataset_dir = '/content/drive/MyDrive/archive-20241008T130234Z-001/archive/brain_tumor_dataset'

# Data generators for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Reserve 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


# Load VGG16 model without the top layers (use imagenet weights)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification (yes/no)

# Define the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])




In [ ]:
# Check the first batch from the training generator
x_train_batch, y_train_batch = train_generator[0]  # Get the first batch
print(f'Training batch shape: {x_train_batch.shape}, Labels shape: {y_train_batch.shape}')

# Check the first batch from the validation generator
x_val_batch, y_val_batch = validation_generator[0]  # Get the first batch
print(f'Validation batch shape: {x_val_batch.shape}, Labels shape: {y_val_batch.shape}')


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10  # Adjust the number of epochs as needed
)


In [ ]:
# Save the model
model.save('/content/drive/My Drive/brain_tumor_model.h5')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/content/drive/My Drive/brain_tumor_model.h5')

# Function to predict if an image has a tumor
def predict_tumor(image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))  # Load the image
    img_array = img_to_array(img) / 255.0  # Convert to array and normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = (prediction > 0.5).astype("int32")  # Threshold for binary classification

    # Determine the class label
    class_labels = ['No Tumor', 'Tumor']  # Adjust based on your training labels
    return class_labels[predicted_class[0][0]], prediction[0][0]

# Example usage
image_path = '/content/drive/MyDrive/archive-20241008T130234Z-001/archive/brain_tumor_dataset/yes/Y1.jpg'
 # Replace with the path to your image
predicted_label, confidence = predict_tumor(image_path)

# Display the result
print(f'Predicted Label: {predicted_label}, Confidence: {confidence:.2f}')

# Optionally, show the image
img = load_img(image_path, target_size=(224, 224))
plt.imshow(img)
plt.title(f'Predicted: {predicted_label}, Confidence: {confidence:.2f}')
plt.axis('off')
plt.show()


In [ ]:
# Example usage
image_path1 = '/content/drive/MyDrive/archive-20241008T130234Z-001/archive/brain_tumor_dataset/no/no 99.jpg'
 # Replace with the path to your image
predicted_label, confidence = predict_tumor(image_path1)

# Display the result
print(f'Predicted Label: {predicted_label}, Confidence: {confidence:.2f}')

# Optionally, show the image
img = load_img(image_path1, target_size=(224, 224))
plt.imshow(img)
plt.title(f'Predicted: {predicted_label}, Confidence: {confidence:.2f}')
plt.axis('off')
plt.show()


In [ ]:
# Example usage
image_path1 = '/content/drive/MyDrive/archive-20241008T130234Z-001/archive/brain_tumor_dataset/no/no 99.jpg'
 # Replace with the path to your image
predicted_label, confidence = predict_tumor(image_path1)

# Display the result
print(f'Predicted Label: {predicted_label}, Confidence: {confidence:.2f}')

# Optionally, show the image
img = load_img(image_path1, target_size=(224, 224))
plt.imshow(img)
plt.title(f'Predicted: {predicted_label}, Confidence: {confidence:.2f}')
plt.axis('off')
plt.show()


In [ ]:
# Example usage
image_path1 = '/content/no tumor image.jpg'
 # Replace with the path to your image
predicted_label, confidence = predict_tumor(image_path1)

# Display the result
print(f'Predicted Label: {predicted_label}, Confidence: {confidence:.2f}')

# Optionally, show the image
img = load_img(image_path1, target_size=(224, 224))
plt.imshow(img)
plt.title(f'Predicted: {predicted_label}, Confidence: {confidence:.2f}')
plt.axis('off')
plt.show()


In [ ]:
# Example usage
image_path1 = '/content/tumor image.jpeg'
 # Replace with the path to your image
predicted_label, confidence = predict_tumor(image_path1)

# Display the result
print(f'Predicted Label: {predicted_label}, Confidence: {confidence:.2f}')

# Optionally, show the image
img = load_img(image_path1, target_size=(224, 224))
plt.imshow(img)
plt.title(f'Predicted: {predicted_label}, Confidence: {confidence:.2f}')
plt.axis('off')
plt.show()